In [1]:
import sys
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel import model
from seqmodel import data

In [26]:
tf.reset_default_graph()
model_opt = model.seq_model.BasicSeqModel.default_opt()
model_opt.loss_type = 'mse'
model_opt.output_mode = 'logit'
model_opt.decoder.rnn_opt.logit.out_vocab_size = 1
model_fn = model.seq_model.BasicSeqModel(model_opt)
lm = model_fn(is_training=True)
optimizer = tf.train.AdamOptimizer()
g_v_pairs =  optimizer.compute_gradients(lm.training_loss, tf.trainable_variables())
train_op = optimizer.apply_gradients(g_v_pairs)

In [27]:
vocab = data.Vocabulary.from_vocab_file('../data/tiny_single/vocab.txt')
valid_iter = data.TokenIterator(vocab, vocab)
valid_iter.initialize('../data/tiny_single/valid.txt')
valid_iter.init_batch(4)

In [28]:
sess_config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())
batch = valid_iter.next_batch()
_l = batch.labels
labels = _l._replace(label=_l.label.astype(np.float32))
batch = batch._replace(labels=labels)
state = None

In [48]:
prediction, state, info = lm.predict(sess, batch.features, state=state, new_seq=True)

In [30]:
eval_loss, state, info = lm.evaluate(sess, batch)

In [46]:
eval_loss, train_loss, state, info = lm.train(sess, batch, train_op)

In [47]:
eval_loss

55.088478

In [51]:
prediction.shape

(10, 4, 1)

In [50]:
batch

SeqTuple(features=SeqFeatureTuple(inputs=array([[ 0, 14,  0,  5],
       [ 6, 12, 12,  6],
       [ 7, 12, 11,  7],
       [ 7,  0, 14,  9],
       [12,  5, 11, 12],
       [10,  8,  0,  0],
       [14,  6,  6,  5],
       [ 0, 12, 10,  5],
       [ 5, 10, 11,  6],
       [ 7, 12,  0,  9]], dtype=int32), input_seq_len=array([10, 10, 10, 10], dtype=int32)), labels=SeqLabelTuple(label=array([[  6.,  12.,  12.,   6.],
       [  7.,  12.,  11.,   7.],
       [  7.,   0.,  14.,   9.],
       [ 12.,   5.,  11.,  12.],
       [ 10.,   8.,   0.,   0.],
       [ 14.,   6.,   6.,   5.],
       [  0.,  12.,  10.,   5.],
       [  5.,  10.,  11.,   6.],
       [  7.,  12.,   0.,   9.],
       [  7.,  14.,   5.,  12.]], dtype=float32), label_weight=array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,